In [18]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarMDA
prob = om.Problem()
prob.model = SellarMDA()
prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-8)

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

prob.setup()
prob.model.list_inputs()

11 Input(s) in 'model'

varname   val  
--------  -----
cycle
  d1
    z     |0.0|
  d2
    z     |0.0|
obj_cmp
  z       |0.0|
cycle
  d1
    x     [0.] 
obj_cmp
  x       [0.] 
cycle
  d1
    y2    [1.] 
obj_cmp
  y2      [1.] 
con_cmp2
  y2      [1.] 
cycle
  d2
    y1    [1.] 
obj_cmp
  y1      [1.] 
con_cmp1
  y1      [1.] 




[('cycle.d1.z', {'val': array([0., 0.])}),
 ('cycle.d2.z', {'val': array([0., 0.])}),
 ('obj_cmp.z', {'val': array([0., 0.])}),
 ('cycle.d1.x', {'val': array([0.])}),
 ('obj_cmp.x', {'val': array([0.])}),
 ('cycle.d1.y2', {'val': array([1.])}),
 ('obj_cmp.y2', {'val': array([1.])}),
 ('con_cmp2.y2', {'val': array([1.])}),
 ('cycle.d2.y1', {'val': array([1.])}),
 ('obj_cmp.y1', {'val': array([1.])}),
 ('con_cmp1.y1', {'val': array([1.])})]

In [16]:
from ipytree import Tree, Node
from ipywidgets import Button, Label, FloatText, HBox, VBox, Text, Output, Layout
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets



In [30]:
from openmdao.core.component import Component
def system_iter(sys, node):
    if sys.name == '_auto_ivc':
        return
    name = sys.name if sys.name else 'root'
    new_node = Node(sys.name)
    node.add_node(new_node)

    if isinstance(sys, Component):
        inputs = list(sys._var_allprocs_prom2abs_list['input'].keys())
    #     print(sys.name, inputs)
        for input in inputs:
            input_node = Node(input)
            new_node.add_node(input_node)
    else:
        for s in sys._subsystems_myproc:
            system_iter(s, new_node)
tree = Tree(stripes=True, multiple_selection=True)
system_iter(prob.model, tree)
tree

Tree(nodes=(Node(name='', nodes=(Node(name='cycle', nodes=(Node(name='d1', nodes=(Node(name='z'), Node(name='x…

In [15]:
for system in prob.model.system_iter(include_self=True, recurse=True):
    system.list_inputs(hierarchical=False)
    
# #     inputs = system.get_io_metadata(('input',))
# #     outputs = system.get_io_metadata(('input',))
# #     print(inputs)
# #     print(system.name, ':', system._var_allprocs_prom2abs_list['output'], '\n')
#     print(system.name, ':', list(system._var_allprocs_prom2abs_list['output'].keys()), '\n')

11 Input(s) in 'model'


varname,val
cycle.d1.z,[0. 0.]
cycle.d2.z,[0. 0.]
obj_cmp.z,[0. 0.]
cycle.d1.x,[0.]
obj_cmp.x,[0.]
cycle.d1.y2,[1.]
obj_cmp.y2,[1.]
con_cmp2.y2,[1.]
cycle.d2.y1,[1.]
obj_cmp.y1,[1.]


5 Input(s) in 'cycle'


varname,val
cycle.d1.z,[0. 0.]
cycle.d2.z,[0. 0.]
cycle.d1.x,[0.]
cycle.d1.y2,[1.]
cycle.d2.y1,[1.]


3 Input(s) in 'cycle.d1'


...

varname,val
obj_cmp.x,[0.]
obj_cmp.y1,[1.]
obj_cmp.y2,[1.]
obj_cmp.z,[0. 0.]


1 Input(s) in 'con_cmp1'


varname,val
con_cmp1.y1,1


1 Input(s) in 'con_cmp2'


varname,val
con_cmp2.y2,1
